In [1]:
import json
import csv
import itertools
import pandas as pd
import numpy as np

In [2]:
filename = "data.json"

In [3]:
df = pd.read_json(filename)

In [7]:
# distinct id of the users who has added item to the cart
user_distinct_ids = df[df.event == "Add to Cart Web"]

In [8]:
user_distinct_ids.head()

,browser,browser_version,city,day_advance,device,distinct_id,duration_of_booking,event,initial_referrer,parent_id,price,referrer,time,utm_campaign,utm_medium,utm_source
0,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES05834,1999,http://flyrobe.com/women/lehengas?delivery_dat...,1503533058,NaN,NaN,NaN
1,Firefox,54.0,Mumbai,96,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533208,NaN,NaN,NaN
2,Firefox,54.0,Mumbai,107,NaN,1573d7c483eb3-06235568306ea58-4c594331-100200-...,4.0,Add to Cart Web,https://www.facebook.com/,FRES01768,2699,http://flyrobe.com/?gclid=Cj0KEQjwz_TMBRD0jY-R...,1503533219,NaN,NaN,NaN
3,Mobile Safari,10.0,Mumbai,10,iPhone,15c82f54f9017e-0dc211ce35bbbc-64793c7f-3d10d-1...,4.0,Add to Cart Web,https://www.google.co.in/,FRES05560,3899,https://flyrobe.com/women/ethnic-gowns,1503533525,NaN,NaN,NaN
4,Chrome,59.0,Delhi,6,Android,15cb6e4205211-08b6c197de6bc7-3f112b20-38400-15...,4.0,Add to Cart Web,https://www.google.co.in/,FRES03765,1499,https://www.google.co.in/,1503536564,NaN,NaN,NaN


In [9]:
user_distinct_ids = user_distinct_ids['distinct_id'].tolist()

In [10]:
user_distinct_ids[0]

'1573d7c483eb3-06235568306ea58-4c594331-100200-1573d7c484011c'

In [11]:
user_events = df[df.distinct_id.isin(user_distinct_ids)]

In [12]:
user_events = user_events.sort_values(by=['time'])

In [13]:
user_events.head()

,browser,browser_version,city,day_advance,device,distinct_id,duration_of_booking,event,initial_referrer,parent_id,price,referrer,time,utm_campaign,utm_medium,utm_source
1591546,Chrome,56.0,Delhi,NaN,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,NaN,Details Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512008,NaN,NaN,NaN
80557,Chrome,56.0,Delhi,22,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,8.0,Add to Cart Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512045,NaN,NaN,NaN
420251,Mobile Safari,10.0,Mumbai,NaN,iPad,blissglobalkidzee@gmail.com,NaN,Details Web,$direct,FRES01492,149,NaN,1486512054,NaN,NaN,NaN
1239440,Chrome,56.0,Delhi,NaN,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,NaN,Details Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512056,NaN,NaN,NaN
769396,Chrome,56.0,Delhi,NaN,NaN,15a19d486641d-05e311073b4267-57e1b3c-100200-15...,NaN,Details Web,https://www.google.co.in/,FRES02262,399,http://flyrobe.com/men/western-jackets,1486512058,NaN,NaN,NaN


In [14]:
# with open('user_events.json', 'w') as f:
#     f.write(df.to_json(orient='records', lines=True))

OSError: [Errno 28] No space left on device

In [16]:
list1 = []; user_dict = {}
for index, each in user_events.iterrows():
    if each["distinct_id"] not in user_dict:
        user_dict[each["distinct_id"]] = [{each["event"]: [each["time"], each["parent_id"]]}]
    else:
        user_dict[each["distinct_id"]].append({each["event"]: [each["time"], each["parent_id"]]})

In [17]:
with open('user_event_data_21-02-2018.json', 'wt') as jsonfile:
    json.dump(user_dict, jsonfile)

In [18]:
# TODO: see for cases (need to check for the validity of the events) ['FRES06022'],
cycle = {}
cycles_for_correlation = []
for user, ev_tm_pi in user_dict.items():
    temp_cycle = []
    for events in ev_tm_pi:
        for key, value in events.items():
            temp_cycle.append(value[1])
            if key == "Add to Cart Web":
                try:
                    cycle[user].append(temp_cycle)
                except KeyError:
                    cycle[user] = [temp_cycle]
                cycles_for_correlation.append(temp_cycle)
                temp_cycle = []

In [19]:
cycles_for_correlation_unique = []

for element in cycles_for_correlation:
    temp_list = []
    for item in list(set(element[:-1])):
        temp_list.append(item)
    temp_list.append(element[-1])
    cycles_for_correlation_unique.append(temp_list)
        


cycles_for_correlation_unique

[['FRES02262', 'FRES02262'],
 ['FRES02262', 'FRES02262'],
 ['FRES04581',
  'FRES01498',
  'FRES03785',
  'FRES01737',
  'FRES01998',
  'FRES01548',
  'FRES03514',
  'FRES01733',
  'FRES04214',
  'FRES01494',
  'FRES01492',
  'FRES03514'],
 ['FRES05064',
  'FRES04581',
  'FRES01498',
  'FRES04587',
  'FRES04753',
  'FRES01496',
  'FRES03860',
  'FRES03785',
  'FRES01737',
  'FRES01733',
  'FRES01678',
  'FEXP03725',
  'FRES01510',
  'FEXP00698',
  'FRES01683',
  'FRES04214',
  'FRES01775',
  'FRES03771',
  'FRES05553',
  'FRES01467',
  'FRES05269',
  'FRES04961',
  'FRES01775'],
 ['FRES01851',
  'FRES05264',
  'FRES05269',
  'FRES01668',
  'FRES01775',
  'FRES05269'],
 ['FRES04215',
  'FRES05553',
  'FRES05364',
  'FRES03514',
  'FRES01733',
  'FRES05269',
  'FRES01733'],
 ['FRES04760', 'FRES01733', 'FRES04760'],
 ['FRES04742',
  'FRES03248',
  'FRES05532',
  'FRES05364',
  'FRES04965',
  'FEXP01025',
  'FRES05574',
  'FRES01848',
  'FRES05784',
  'FRES04986',
  'FEXP01038',
  'FRES0567

In [20]:
cycles_for_correlation_unique_updated = []
for cycle in cycles_for_correlation_unique:
    if len(cycle) != 1:
        cycles_for_correlation_unique_updated.append(cycle)

In [21]:
#cycles_for_correlation_unique_updated = cycles_for_correlation_unique_updated[:10]

In [22]:



count = 0
for element in cycles_for_correlation_unique_updated:
    if len(element) ==1:
        count += 1
count

0

In [23]:
look_up_men = []
look_up_women = []
with open('parent_item_data_female_21-02-2018.json', 'r') as f:
     look_up_women = json.load(f)
        
with open('parent_item_data_male_21-02-2018.json', 'r') as f:
     look_up_men = json.load(f)

unique_inven_cat_men = []
unique_brand_ids_men = []
for element in look_up_men.keys():
    if look_up_men[element]["inventory_category"] not in unique_inven_cat_men:
        if look_up_men[element]["inventory_category"] != None:
            unique_inven_cat_men.append(look_up_men[element]["inventory_category"])
        
    if look_up_men[element]["brand_id"] not in unique_brand_ids_men:
        if look_up_men[element]["brand_id"] != None:
            unique_brand_ids_men.append(look_up_men[element]["brand_id"])
        
        
unique_inven_cat_women = []
unique_brand_ids_women = []
for element in look_up_women.keys():
    if look_up_women[element]["inventory_category"] not in unique_inven_cat_women:
        if look_up_women[element]["inventory_category"] != None:
            unique_inven_cat_women.append(look_up_women[element]["inventory_category"])
        
    if look_up_women[element]["brand_id"] not in unique_brand_ids_women:
        if look_up_women[element]["brand_id"]  != None:
            unique_brand_ids_women.append(look_up_women[element]["brand_id"])



            
# later update this lookup
# fetch rentals from rds 
#unique_inven_cat_women
#look_up_men

In [24]:
unique_inven_cat_men

[23, 29, 28, 36, 37, 24, 30, 38, 5, 40, 10]

In [25]:
# get all unique inventory_categories and brand_ids
unique_inven_cat_men = list(set(unique_inven_cat_men))
unique_brand_ids_men = list(set(unique_brand_ids_men))
inventory_cat_correlation_matrix_men = np.identity(len(unique_inven_cat_men))
brand_correlation_matrix_men = np.identity(len(unique_brand_ids_men))

unique_inven_cat_women = list(set(unique_inven_cat_women))
unique_brand_ids_women = list(set(unique_brand_ids_women))
inventory_cat_correlation_matrix_women = np.identity(len(unique_inven_cat_women))
brand_correlation_matrix_women = np.identity(len(unique_brand_ids_women))

In [26]:
#look_up_men

In [27]:
tot_occ_map_men = {}
tot_occ_map_women = {}

# TODO: make different cor-mat for male/female

inventory_category_cycle_men = []
inventory_category_cycle_women = []

for cycle in cycles_for_correlation_unique_updated:
    try:
        # inventory category exists in our lookup or not
        local_list = []
        for element in cycle:
            if look_up_men[element]['inventory_category']:
                local_list.append(look_up_men[element]['inventory_category'])
        inventory_category_cycle_men.append(local_list)
    except KeyError:
        pass


for cycle in cycles_for_correlation_unique_updated:
    try:
        # inventory category exists in our lookup or not
        local_list = []
        for element in cycle:
            if look_up_women[element]['inventory_category']:
                local_list.append(look_up_women[element]['inventory_category'])
        inventory_category_cycle_women.append(local_list)
    except KeyError:
        pass


for cycle in inventory_category_cycle_men:
    for element in cycle[:-1]:
        if element not in tot_occ_map_men.keys() and element != None and element not in [6,7,8,22,25,26,27,31,32,39,45,46]:
            tot_occ_map_men[element] = 1
        elif element != None and element != None and element not in [6,7,8,22,25,26,27,31,32,39,45,46]:
            tot_occ_map_men[element] += 1

for cycle in inventory_category_cycle_women:
    for element in cycle[:-1]:
        if element not in tot_occ_map_women.keys() and element != None and element not in [6,7,8,22,25,26,27,31,32,39,45,46]:
            tot_occ_map_women[element] = 1
        elif element != None and element != None and element not in [6,7,8,22,25,26,27,31,32,39,45,46]:
            tot_occ_map_women[element] += 1



# cl = []
# for item in cycles_for_correlation_unique_updated:
#     for element in item[:-1]:
#         if element in look_up_men.keys():
#             if (look_up_men[element]['inventory_category'] in tot_occ_map_men.keys()) and look_up_men[element]['inventory_category'] != None and look_up_men[element]['inventory_category'] not in [6,7,8,22,25,26,27,31,32,39,45,46]:
#                 tot_occ_map_men[look_up_men[element]['inventory_category']] += 1
#             elif look_up_men[element]['inventory_category'] != None and look_up_men[element]['inventory_category'] not in [6,7,8,22,25,26,27,31,32,39,45,46]:
#                 tot_occ_map_men[look_up_men[element]['inventory_category']] = 1
#         elif element in look_up_women.keys():
#             #print(element)
#             if (look_up_women[element]['inventory_category'] in tot_occ_map_women.keys()) and look_up_women[element]['inventory_category'] != None and look_up_women[element]['inventory_category'] not in [6,7,8,22,25,26,27,31,32,39,45,46]:
#                 tot_occ_map_women[look_up_women[element]['inventory_category']] += 1
#             elif look_up_women[element]['inventory_category'] != None and look_up_women[element]['inventory_category'] not in [6,7,8,22,25,26,27,31,32,39,45,46]:
#                 tot_occ_map_women[look_up_women[element]['inventory_category']] = 1
                

#tot_occ_map_women

In [28]:
#tot_occ_map_men

In [29]:
#cycles_for_correlation_unique_updated

In [30]:
corelation_dict_ic_men = {}
unique_inven_cat_men = tot_occ_map_men.keys()
for element in sorted(unique_inven_cat_men):
    corelation_dict_ic_men[element] = {}
    for inner_element in sorted(unique_inven_cat_men):
        #if inner_element != element:
        corelation_dict_ic_men[element][inner_element] = 0


unique_inven_cat_women = tot_occ_map_women.keys()
corelation_dict_ic_women = {}
for element in sorted(unique_inven_cat_women):
    corelation_dict_ic_women[element] = {}
    for inner_element in sorted(unique_inven_cat_women):
        #if inner_element != element:
        corelation_dict_ic_women[element][inner_element] = 0


        
# corelation_dict
    

In [31]:

# cycles_for_correlation_unique_cat_men = []
# test_blazzers = 0
# cycles_for_correlation_unique_cat_women = []
# for element in cycles_for_correlation_unique:
#     for item in element[:-1]:
#         try:
#             if look_up_men[item]['inventory_category']:
#                 cycles_for_correlation_unique_cat_men.append(look_up_men[item]['inventory_category'])
#         except KeyError:
#             try:
#                 if look_up_women[item]['inventory_category']:
#                     cycles_for_correlation_unique_cat_women.append(look_up_women[item]['inventory_category'])
#             except KeyError:
#                 pass

#     try:
#         if look_up_men[element[-1]]['inventory_category']:
#             cycles_for_correlation_unique_cat_men.append(look_up_men[element[-1]]['inventory_category'])
#     except KeyError:
#         try:
#             if look_up_women[element[-1]]['inventory_category']:
#                 cycles_for_correlation_unique_cat_women.append(look_up_women[element[-1]]['inventory_category'])
#         except KeyError:
#             pass
        
# cycles_for_correlation_unique_cat_men.count(28), test_blazzers
#look_up_women

In [32]:
# for item in cycles_for_correlation:
#     for keys, values in corelation_dict.items():
#         for val in values.keys():
            
#             if corelation_dict[keys][val] == look_up[item[-1]]['inventory_category']:
#                 pass
#             else:
#                 corelation_dict[keys][look_up[item[-1]]['inventory_category']] += 1
#             try:
#                 if look_up[item[-1]]:
#                     corelation_dict[look_up[item[-1]]['inventory_category']] += 1
#             except KeyError:
#                 try:
#                     corelation_dict[look_up[item[-1]]['inventory_category']] = 1
#                 except KeyError:
#                     pass
count = 0
found = 0



print("Men")

for cycle in inventory_category_cycle_men:
    for key, value in corelation_dict_ic_men.items():
        for inner_key, inner_val in value.items():
            if inner_key == cycle[-1]:
                for element in cycle[:-1]:
                    if key == element:
                        #print(key, element, inner_key)
                        corelation_dict_ic_men[key][inner_key] += 1

# for cycle in cycles_for_correlation_unique_updated:
#     try:
#         if look_up_men[cycle[-1]]['inventory_category']:
#             for key, value in corelation_dict_ic_men.items():
#                 for inner_key in values.keys():
#                     if inner_key == look_up_men[cycle[-1]]['inventory_category']:
#                         for element in cycle[:-1]:
#                             if key == look_up_men[element]['inventory_category']:
#                                 #print(key, element, inner_key)
#                                 corelation_dict_ic_men[key][inner_key] += 1
#     except KeyError:
#         pass
#         #print(cycle[-1])
    
print("Women")

for cycle in inventory_category_cycle_women:
    for key, value in corelation_dict_ic_women.items():
        for inner_key, inner_val in value.items():
            if inner_key == cycle[-1]:
                for element in cycle[:-1]:
                    if key == element:
                        #print(key, element, inner_key)
                        corelation_dict_ic_women[key][inner_key] += 1


# for cycle in cycles_for_correlation_unique_updated:
#     try:
#         if look_up_women[cycle[-1]]['inventory_category']:
#             for key, value in corelation_dict_ic_women.items():
#                 for inner_key in values.keys():
#                     if inner_key == look_up_women[cycle[-1]]['inventory_category']:
#                         for element in cycle[:-1]:
#                             if key == look_up_women[element]['inventory_category']:
#                                 #print(key, element, inner_key)
#                                 corelation_dict_ic_women[key][inner_key] += 1
#     except KeyError:
#         pass
        #print(cycle[-1])
                
# for key, value in corelation_dict_ic_men.items(): #[ A: [A:, B:, C:, D:]] [iB1, i2 i3...i10]
#     for inner_key in value.keys():
#         for item in cycles_for_correlation_unique_updated:
#             #if key != inner_key:
#             #if item[-1] in look_up_men:
#             try:
#                 if inner_key == look_up_men[item[-1]]['inventory_category']:
#                     if inner_key == 29: 
#                         print(item[-1])
#                     for element in item[:-1]:
#                         if key == look_up_men[element]['inventory_category']:
#                             #print(key, element, inner_key)
#                             corelation_dict_ic_men[key][inner_key] += 1
#             except KeyError:
#                 pass
            
# for key, value in corelation_dict_ic_women.items(): #[ A: [A:, B:, C:, D:]] [iB1, i2 i3...i10]
#     for inner_key in value.keys():
#         for item in cycles_for_correlation_unique_updated:
#             #if key != inner_key:
#             #if item[-1] in look_up_men:
#             try:
#                 if inner_key == look_up_women[item[-1]]['inventory_category']:
#                     #print(item[-1], key, inner_key)
#                     for element in item[:-1]:
#                         if key == look_up_women[element]['inventory_category']:
#                             #print(key, element, inner_key)
#                             corelation_dict_ic_women[key][inner_key] += 1
#             except KeyError:
#                 pass
                #print(item[-1], key, inner_key)
            
#             elif item[-1] in look_up_women:
#                 if inner_key == look_up_women[item[-1]]['inventory_category']:
#                     for element in item[:-1]:
#                         if key == look_up_women[element]['inventory_category']:
#                             corelation_dict_ic_women[key][inner_key] += 1
#             try:
#                 if inner_key == look_up[item[-1]]['inventory_category']:
#                     for element in item[:-1]:
#                         if key == look_up[element]['inventory_category']:
#                             corelation_dict[key][inner_key] += 1
#                 found += 1
#             except KeyError:
#                 count += 1

Men
Women


In [33]:
#count, found
#corelation_dict_ic_women

In [34]:
for key, values in corelation_dict_ic_women.items():
    for inner_key in values.keys():
        try:
            #print(type(key), type(inner_key))
            #print(corelation_dict_ic_women[key][inner_key], tot_occ_map_women[inner_key], key, inner_key)
            corelation_dict_ic_women[key][inner_key] = float("%.4f" % (corelation_dict_ic_women[key][inner_key] / tot_occ_map_women[key]))
        except KeyError:
            pass
        
for key, values in corelation_dict_ic_men.items():
    for inner_key in values.keys():
        try:
            #print(corelation_dict_ic_men[key][inner_key], tot_occ_map_men[inner_key], tot_occ_map_men[key], inner_key)
            corelation_dict_ic_men[key][inner_key] = float("%.4f" % (corelation_dict_ic_men[key][inner_key] / tot_occ_map_men[key]))
#             if corelation_dict_ic_men[key][inner_key] > 1:
#                 print(corelation_dict_ic_men[key][inner_key], tot_occ_map_men[inner_key], tot_occ_map_men[key], key, inner_key)
        except KeyError:
            pass

In [35]:
#corelation_dict_ic_men, tot_occ_map_men

In [36]:
#corelation_dict_ic_women
for element, inner_key in corelation_dict_ic_men.items():
    print(inner_key, sum(list(inner_key.values())))
#     if round(sum(list(inner_key.values()))) != 1:
#         print(inner_key.values(), element, inner_key)

    


{23: 0.5668, 24: 0.0334, 28: 0.0603, 29: 0.1187, 30: 0.0818, 36: 0.096, 37: 0.0302, 38: 0.0126, 40: 0.0002} 1.0
{23: 0.0311, 24: 0.648, 28: 0.0221, 29: 0.0739, 30: 0.0357, 36: 0.1439, 37: 0.0401, 38: 0.0048, 40: 0.0005} 1.0001
{23: 0.0248, 24: 0.0064, 28: 0.8203, 29: 0.0445, 30: 0.065, 36: 0.0185, 37: 0.0041, 38: 0.0164, 40: 0.0001} 1.0001
{23: 0.0365, 24: 0.0222, 28: 0.0427, 29: 0.7691, 30: 0.0324, 36: 0.0602, 37: 0.0177, 38: 0.0189, 40: 0.0004} 1.0001000000000002
{23: 0.0421, 24: 0.0098, 28: 0.1005, 29: 0.0431, 30: 0.7539, 36: 0.0338, 37: 0.0074, 38: 0.0094, 40: 0.0} 1.0
{23: 0.0594, 24: 0.092, 28: 0.0348, 29: 0.0929, 30: 0.0436, 36: 0.6159, 37: 0.0464, 38: 0.0147, 40: 0.0004} 1.0001
{23: 0.0587, 24: 0.11, 28: 0.027, 29: 0.0947, 30: 0.0428, 36: 0.1512, 37: 0.5056, 38: 0.0095, 40: 0.0005} 0.9999999999999999
{23: 0.0384, 24: 0.0161, 28: 0.1007, 29: 0.123, 30: 0.0477, 36: 0.0661, 37: 0.0077, 38: 0.6003, 40: 0.0} 1.0
{23: 0.0128, 24: 0.0513, 28: 0.0256, 29: 0.1282, 30: 0.0, 36: 0.0385, 3

In [37]:
corelation_dict_ic_women

{1: {1: 0.6266,
  2: 0.0204,
  3: 0.0528,
  4: 0.0987,
  5: 0.0132,
  9: 0.0343,
  10: 0.0042,
  11: 0.0267,
  12: 0.0071,
  13: 0.0001,
  14: 0.0001,
  15: 0.0407,
  16: 0.0068,
  17: 0.001,
  18: 0.026,
  19: 0.0349,
  20: 0.0017,
  21: 0.0,
  34: 0.0,
  35: 0.0045,
  47: 0.0},
 2: {1: 0.2349,
  2: 0.312,
  3: 0.078,
  4: 0.1151,
  5: 0.0223,
  9: 0.0269,
  10: 0.0084,
  11: 0.0371,
  12: 0.0056,
  13: 0.0009,
  14: 0.0,
  15: 0.0576,
  16: 0.0065,
  17: 0.0009,
  18: 0.0362,
  19: 0.0501,
  20: 0.0009,
  21: 0.0009,
  34: 0.0,
  35: 0.0056,
  47: 0.0},
 3: {1: 0.2076,
  2: 0.0221,
  3: 0.5239,
  4: 0.0573,
  5: 0.0402,
  9: 0.0171,
  10: 0.0086,
  11: 0.0162,
  12: 0.0054,
  13: 0.0009,
  14: 0.0005,
  15: 0.0338,
  16: 0.005,
  17: 0.0005,
  18: 0.019,
  19: 0.0343,
  20: 0.0041,
  21: 0.0005,
  34: 0.0014,
  35: 0.0018,
  47: 0.0},
 4: {1: 0.0315,
  2: 0.0029,
  3: 0.0035,
  4: 0.5692,
  5: 0.0011,
  9: 0.1518,
  10: 0.0088,
  11: 0.1231,
  12: 0.0141,
  13: 0.0017,
  14: 0.0072,


In [38]:
import csv
corelation_dict_ic_men_list = []
corelation_dict_ic_women_list = []
corelation_dict_ic_men_list = list(sorted(corelation_dict_ic_men.keys()))
corelation_dict_ic_women_list = list(sorted(corelation_dict_ic_women.keys()))
    

with open('men_inventory_category_correlation_matrix.csv', 'wt') as myfile:
    wr = csv.writer(myfile, delimiter=',', quoting=csv.QUOTE_ALL)
    #print(corelation_dict_ic_men_list)
    corelation_dict_ic_men_list = [0] + corelation_dict_ic_men_list
    wr.writerow(corelation_dict_ic_men_list)
    
    for key, val in sorted(corelation_dict_ic_men.items()):
        inner_list = [key]
        for inner_key, inner_val in sorted(val.items()):
            inner_list.append(inner_val)
        #print(inner_list)
        wr.writerow(inner_list)

        
with open('women_inventory_category_correlation_matrix.csv', 'wt') as myfile:
    wr = csv.writer(myfile, delimiter=',', quoting=csv.QUOTE_ALL)
    #print(corelation_dict_ic_men_list)
    corelation_dict_ic_women_list = [0] + corelation_dict_ic_women_list
    wr.writerow(corelation_dict_ic_women_list)
    
    for key, val in sorted(corelation_dict_ic_women.items()):
        inner_list = [key]
        for inner_key, inner_val in sorted(val.items()):
            inner_list.append(inner_val)
        #print(inner_list)
        wr.writerow(inner_list)


In [39]:
for element, inner_key in corelation_dict_ic_men.items():
    print(sum(list(inner_key.values())))
    
for element, inner_key in corelation_dict_ic_women.items():
    print(sum(list(inner_key.values())))
    


1.0
1.0001
1.0001
1.0001000000000002
1.0
1.0001
0.9999999999999999
1.0
0.9999
0.9998
0.9999
1.0001999999999998
0.9998999999999998
1.0000999999999998
0.9999000000000002
1.0000000000000002
1.0000999999999998
1.0
0.9997999999999999
1.0002
1.0000999999999998
1.0001
0.9999000000000001
1.0000000000000002
1.0
0.9998999999999998
1.0000000000000002
1.0002
1.0001000000000002
1.0


In [29]:
# list_of_cat   A    B     C

# A   1
# B       1
# C           1
# inv_cat_dict = {}
# for counter, each in enumerate(inv_cat_list):
#     #print(type(each[0]), type(each))
#     if len(each) > 1:
#         try:
#             inv_cat_dict[each[0]].append(each[1])
#         except Exception:
#             inv_cat_dict[each[0]] = [each[1]]
        #print(each, type(each))

#     if each[0] in inv_cat_dict:
#         try:
#             inv_cat_dict[each[0]].append(each[1])
#         except:
#             print(each, counter)
#             break
#     else:
#         inv_cat_dict[each[0]] = [each[1]]
        
# brand_dict = {}
# for each in brand_id_list:
#     if each in brand_dict:
#         brand_dict[each[0]].append(each[1])
#     else:
#         brand_dict[each[0]] = [each[1]]


In [30]:
inv_cat_dict

NameError: name 'inv_cat_dict' is not defined

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [ ]:
# with open('/home/flyrobe/Documents/elasticsearch_to_json/bronze/inventory_category.json', 'w') as outfile:
#     json.dump(inv_cat_dict, outfile)

In [17]:
#user_grouped_events = user_events.groupby( [ "distinct_id", "event", "time", "parent_id"] )

In [40]:
#user_grouped_events = user_grouped_events[['event','distinct_id', 'time', 'parent_id']]

In [1]:
#user_grouped_events.size()

NameError: name 'user_grouped_events' is not defined

In [31]:
import pandas
import numpy

dtype = [('Col1','int32'), ('Col2','float32'), ('Col3','float32')]
values = numpy.zeros(20, dtype=dtype)
index = ['Row'+str(i) for i in range(1, len(values)+1)]

df = pandas.DataFrame(values, index=index)

In [32]:
df.head()

,Col1,Col2,Col3
Row1,0,0.0,0.0
Row2,0,0.0,0.0
Row3,0,0.0,0.0
Row4,0,0.0,0.0
Row5,0,0.0,0.0


In [92]:
float(3.0/37)

0.08108108108108109